In [1]:
import os
import pickle

import pandas as pd
from tqdm.notebook import tqdm
import multiprocess

from agent import CodeAgent

In [2]:
df = pd.concat(
        [pd.read_csv(f"generated/{file}") for file in os.listdir("generated") if file not in ["incomplete_samples.csv"]]
    ).reset_index(drop=True)

df["sample_type"] = None
df["initial_test_case"] = None
df["initial_output"] = None

df.loc[df["coverage"] > 99, "sample_type"] = 1

df = df[df.sample_type == 1].reset_index(drop=True)
df

,original_code,pytest_code,coverage,sample_type,initial_test_case,initial_output
0,"def quaternion_multiply(r, q):\r\n \r\n ...",# test_source.py\r\nimport pytest\r\nimport so...,100.0,1,None,None
1,"def sing_three(mu, c, i0=1.0):\r\n \r\n ...",import pytest\r\nfrom source import sing_three...,100.0,1,None,None
2,def get_rgb_from_int(rgb_int):\r\n \r\n ...,import pytest\r\nimport sys\r\nsys.path.insert...,100.0,1,None,None
3,"def inflate(tensor, times, dim):\r\n \r\n ...",# test_source.py\r\nimport pytest\r\nfrom sour...,100.0,1,None,None
4,def radii(mag):\r\n \r\n # ADM mask all ...,# test_source.py\r\nimport pytest\r\nimport sy...,100.0,1,None,None
...,...,...,...,...,...,...
51770,"\nimport math\n\ndef calculate_hypotenuse(a, b...",import pytest\nfrom source import calculate_hy...,100.0,1,None,None
51771,"import math\ndef getSum ( a , n ) :\n sum =...",import pytest\nfrom source import *\n\ndef tes...,100.0,1,None,None
51772,"def digitWell ( n , m , k ) :\n cnt = 0\n ...","import pytest\nfrom source import digitWell, f...",100.0,1,None,None
51773,"\ndef find_missing_number(numbers):\n """"""\n...",import pytest\nfrom source import find_missing...,100.0,1,None,None


In [3]:
def generate_incorrect_case(df):

    try:
        from agent import CodeAgent
        from adjustment_utils import run_pytest
        code, pytest_code, *rest = df
        sys_prompt = """You are a spechless coding assistant that helps the user generate training data.
        You will receive code representing a pytest file and you will introduce a small bug that can either:
            - make the code throw an error at runtime.
            - reduce the total code coverage.
        You can implement both points for any given sample.
        You **MUST*** return the full generated file.  
        """
        model = CodeAgent(sys_prompt)
        incomplete_code = model.generate_response(pytest_code, n=1)[0]

        res = run_pytest(code, incomplete_code)

        if (not res["failed_assertions"]) and\
                (res["stderr"] == "") and\
                (res["coverage"] > 99):
            return [None] * 6

        return (code, pytest_code, res["coverage"], 3, incomplete_code, f"{res['stdout']}\n{res['stderr']}")
    except:
        return [None] * 6

In [4]:
step_size = 2000


for i in range(49_000, df.shape[0], step_size):
    cnt_df = df.iloc[i : i + step_size, :].to_numpy().tolist()
    num_tasks = len(cnt_df)
    with multiprocess.Pool(48) as pool:
        results = list(tqdm(pool.imap(generate_incorrect_case, cnt_df), total=num_tasks))
    res = pd.DataFrame(results)
    res.columns = df.columns


    res.to_csv(f"bug_insertion_results/{i}_{i + step_size}_bug_insertion_results.csv", index=False)

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/775 [00:00<?, ?it/s]